# 1. Connect to Azure Machine Learning Registry

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

## 1.1 Import the required libraries

In [ ]:
!pip install -U azure-ai-ml

In [ ]:
# Import required libraries
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

from azure.ai.ml import MLClient, Input, load_component
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml.dsl import pipeline

## 1.2 Configure credential

We are using `DefaultAzureCredential` to get access to workspace. 
`DefaultAzureCredential` should be capable of handling most Azure SDK authentication scenarios. 

Reference for more available credentials if it does not work for you: [configure credential example](../../configuration.ipynb), [azure-identity reference doc](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity?view=azure-python).

In [ ]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

## 1.3 Get a handle to the registry

We need to initialize a MlClient pointed to the registry where the OpenAI components are available. [Check this api documentation for more details](https://learn.microsoft.com/en-us/python/api/azure-ai-ml/azure.ai.ml.mlclient?view=azure-python)

In [ ]:
# Get a handle to registry
ml_client = MLClient(
    credential=credential, registry_name="azureml", registry_location="swedencentral"
)

# 2. Define and create components into workspace
## 2.1 Load components from registry
You need to modify the required params in the yaml files like version, compute uri, etc according to your requirements

In [ ]:
finetune_pipeline = load_component(
    client=ml_client, name="openai_completions_finetune_pipeline", version="0.1.2"
)

## 2.2 Inspect loaded components

In [ ]:
print("Pipeline component loaded successfully. Component spec:")
print(finetune_pipeline)

# 3. Sample pipeline job
## 3.1 Build pipeline

There are two types of tasks :

1. completion : input dataset should have rows with "prompt" and "completion" keys
2. chat : each row in the input dataset is a conversation represented as a list of json objects where each json object has "role" as one of the two values - user, assistant and "content" with the utterance

In [ ]:
TASK_TYPE = "chat"  # Change this to completion for completion dataset


# Construct pipeline
@pipeline()
def pipeline_with_registered_component(
    train_dataset,
    validation_dataset,
    training_max_epochs=20,
    model="gpt-35-turbo",
    registered_model_name="sdk-test-turbo-chat-1",
    learning_rate_multiplier=1,
    batch_size=-1,
    task_type="chat",
):
    openai_completions_finetune_component_results = finetune_pipeline(
        train_dataset=train_dataset,
        validation_dataset=validation_dataset,
        n_epochs=training_max_epochs,
        model=model,
        task_type=task_type,
        registered_model_name=registered_model_name,
        learning_rate_multiplier=learning_rate_multiplier,
        batch_size=batch_size,
    )

    return openai_completions_finetune_component_results


pipeline_job = pipeline_with_registered_component(
    train_dataset=Input(type=AssetTypes.URI_FOLDER, path="./data/"),
    validation_dataset=Input(type=AssetTypes.URI_FOLDER, path="./data/"),
    training_max_epochs=1,
    model="gpt-35-turbo",  # Select any model from ["babbage-002", "davinci-002", "gpt-35-turbo"]
    task_type=TASK_TYPE,
    registered_model_name="sdk-test-turbo-chat-1",
    learning_rate_multiplier=1,
    batch_size=-1,
)

# set pipeline level compute
pipeline_job.settings.default_compute = "serverless"

## 3.2 Configure workspace details and get a handle to the workspace

To connect to a workspace, we need identifier parameters - a subscription, resource group and workspace name. We will use these details in the `MLClient` from `azure.ai.ml` to get a handle to the required Azure Machine Learning workspace. We use the default [default azure authentication](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity.defaultazurecredential?view=azure-python) for this tutorial. Check the [configuration notebook](https://github.com/Azure/azureml-examples/blob/6142c51451561447befa665e8dd6fb3ff80bdb62/sdk/python/jobs/configuration.ipynb) for more details on how to configure credentials and connect to a workspace.

In [ ]:
# Get a handle to workspace
ml_client = None
try:
    ml_client = MLClient.from_config(credential)
except Exception as ex:
    print(ex)
    # Enter details of your AML workspace
    subscription_id = ""
    resource_group = ""
    workspace_name = ""

    ml_client = MLClient(credential, subscription_id, resource_group, workspace_name)

## 3.3 Submit pipeline job

In [ ]:
# Submit pipeline job to workspace
pipeline_job = ml_client.jobs.create_or_update(
    pipeline_job, experiment_name="chat_pipeline_test"
)
pipeline_job

In [ ]:
# Wait until the job completes
ml_client.jobs.stream(pipeline_job.name)

In [ ]:
models = ml_client.models.list(name = 'sdk-test-turbo-chat-1')
for model in models:
    print(model.name)
    print(model.version)

In [ ]:
import requests
import json
import subprocess

# 1. Add your Azure OpenAI account details
subscription = ""
resource_group = ""
resource_name = "" #openai resource name
model_deployment_name = ""

# 2. Add the AzureML registered model name, registered model version, and the AzureML (AML) workspace path for your fine-tuned model.
# Your registered models data can be found in the `Models` tab of your AzureML workspace.
registered_model_name = "sdk-test-turbo-chat-1"
registered_model_version = "1"
workspace_path = f"/subscriptions/{subscription}/resourcegroups/{resource_group}/providers/Microsoft.MachineLearningServices/workspaces/{workspace_name}"

In [ ]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    access_token = credential.get_token("https://management.azure.com/.default")
    token = access_token.token
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    # This will open a browser page for
    credential = InteractiveBrowserCredential()

In [ ]:
deploy_params = {"api-version": "2023-05-01"}
deploy_headers = {
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json",
}

# 4. Set model deployment configuration. Here capacity refers to support for `1K Tokens Per Minute (TPM)` for your deployment.
deploy_data = {
    "sku": {"name": "Standard", "capacity": 1},
    "properties": {
        "model": {
            "format": "OpenAI",
            "name": f"{registered_model_name}",
            "version": f"{registered_model_version}",
            "source": f"{workspace_path}",
        }
    },
}

deploy_data = json.dumps(deploy_data)

In [ ]:
# 5. Send PUT request to Azure cognitive services to create model deployment
request_url = f"https://management.azure.com/subscriptions/{subscription}/resourceGroups/{resource_group}/providers/Microsoft.CognitiveServices/accounts/{resource_name}/deployments/{model_deployment_name}"

r = requests.put(
    request_url, 
    params=deploy_params, 
    headers=deploy_headers, 
    data=deploy_data
)

print(r.json())

# 6. View Your model deployment status in Azure OpenAI Studio portal
# Visit [https://oai.azure.com/portal/](https://oai.azure.com/portal/) to view your model deployments.

# 7. After your deployment succeeds, use Azure OpenAI chat playground to test your Deployment


In [ ]:
# 6. Send DELETE request to Azure cognitive services to delete model deployment
request_url = f"https://management.azure.com//subscriptions/{subscription}/resourceGroups/{resource_group}/providers/Microsoft.CognitiveServices/accounts/{resource_name}/deployments/{model_deployment_name}"
r = requests.delete(
    request_url, 
    params=deploy_params, 
    headers=deploy_headers
)
print(r.json())